<a href="https://colab.research.google.com/github/iko-ai/ibibio-tts/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from glob import glob
import csv
import random
import pandas as pd

In [4]:
print(len(os.listdir("/content/drive/MyDrive/dataset/wav_split/wavs")))

603


In [ ]:
# Paths
input_excel_path = '/content/metadata.xlsx'
output_csv_path = '/content/drive/MyDrive/dataset/metadata.csv'

# Read the Excel file
df = pd.read_excel(input_excel_path, engine='openpyxl')

# Save the dataframe as a CSV file
df.to_csv(output_csv_path, index=False, encoding='utf-8')

print("Excel file has been successfully converted to CSV.")


Excel file has been successfully converted to CSV.


In [ ]:
# Paths
input_csv_path = '/content/drive/MyDrive/dataset/metadata.csv'
input_wavs_dir = '/content/drive/MyDrive/dataset/wav'
output_dir = '/content/drive/MyDrive/dataset/wav_split'
train_output_path = os.path.join(output_dir, 'train.txt')
val_output_path = os.path.join(output_dir, 'val.txt')

In [ ]:
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the input CSV
with open(input_csv_path, 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    data = list(reader)

# Shuffle the data
random.shuffle(data)

# Split the data into 80% training and 20% validation
split_index = int(0.8 * len(data))
train_data = data[:split_index]
val_data = data[split_index:]

# Function to write data to a text file
def write_data(file_path, data):
    with open(file_path, 'w') as file:
        for row in data:
            # Extract just the filename from the full file path
            filename = os.path.basename(row['Filepath'])
            transcription = row['Transcription']
            file.write(f"wavs/{filename}|{transcription}\n")

In [ ]:
# Write the training and validation data to their respective files
write_data(train_output_path, train_data)
write_data(val_output_path, val_data)

In [4]:
!git clone https://github.com/iko-ai/ibibio-tts.git

Cloning into 'ibibio-tts'...
remote: Enumerating objects: 16671, done.
remote: Counting objects: 100% (440/440), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 16671 (delta 244), reused 370 (delta 210), pack-reused 16231
Receiving objects: 100% (16671/16671), 12.17 MiB | 28.85 MiB/s, done.
Resolving deltas: 100% (13357/13357), done.


In [5]:
#%cd ../

In [6]:
#%cd ibibio-texttospeech/codes
#%cd codes
%cd /content/ibibio-tts/codes

/content/ibibio-tts/codes


In [ ]:
!pip install -r requirements.laxed.txt

In [ ]:
!wget https://huggingface.co/Gatozu35/tortoise-tts/resolve/main/dvae.pth -O /content/ibibio-tts/experiments/dvae.pth
!wget https://huggingface.co/jbetker/tortoise-tts-v2/resolve/main/.models/autoregressive.pth -O /content/ibibio-tts/experiments/autoregressive.pth

# Make sure the waveform audio has a sampling rate of 22.05kHz
Install librosa using the following command (automatically installs soundfile as well):
pip install librosa

In [5]:
print(len(os.listdir("/content/drive/MyDrive/dataset/wav_split/wavs")))

603


In [6]:
import librosa
import soundfile as sf

def resample_wav_file(input_file, target_sampling_rate=22050):
    # Load audio file
    audio, sampling_rate = librosa.load(input_file, sr=None)

    # Check if the sampling rate matches the target
    if sampling_rate != target_sampling_rate:

        # Resample audio to the target sampling rate
        audio_resampled = librosa.resample(audio, orig_sr=sampling_rate, target_sr=target_sampling_rate)

        # Overwrite the input file with the resampled audio
        sf.write(input_file, audio_resampled, target_sampling_rate)

In [7]:
# Resample all audio samples to 22.05kHz
dataset_path = '/content/drive/MyDrive/dataset/wav_split/wavs'
for wav_file in glob(dataset_path + "*.wav"):
    resample_wav_file(input_file)

# Fine-Tune THE AUTOREGRESSIVE MODEL

In [ ]:
!python3 /content/ibibio-tts/codes/train.py -opt /content/ibibio-tts/experiments/custom_language_gpt.yml

Disabled distributed training.
Path already exists. Rename it to [/content/ibibio-tts/experiments/custom_language_gpt_archived_240606-225245]
24-06-06 22:52:45.655 - INFO:   name: custom_language_gpt
  model: extensibletrainer
  scale: 1
  gpu_ids: [0]
  start_step: 0
  checkpointing_enabled: True
  fp16: False
  use_8bit: True
  wandb: True
  use_tb_logger: False
  datasets:[
    train:[
      name: train_dataset
      n_workers: 8
      batch_size: 128
      mode: paired_voice_audio
      path: /content/drive/MyDrive/dataset/wav_split/train.txt
      fetcher_mode: ['lj']
      phase: train
      max_wav_length: 255995
      max_text_length: 200
      sample_rate: 22050
      load_conditioning: True
      num_conditioning_candidates: 2
      conditioning_length: 44000
      use_bpe_tokenizer: True
      load_aligned_codes: False
      tokenizer_vocab: /content/drive/MyDrive/dataset/custom_language_tokenizer.json
      data_type: img
    ]
    val:[
      name: val_dataset
      n_work

In [9]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.5/289.5 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install transformers==4.33.2

In [ ]:
import os
print(os.getcwd())


/content/ibibio-tts/codes


In [ ]:
import tokenizers
print(tokenizers.__version__)


0.13.3


In [ ]:
import re
import torch
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer



def remove_extraneous_punctuation(word):
    replacement_punctuation = {
        '{': '(', '}': ')',
        '[': '(', ']': ')',
        '`': '\'', '—': '-',
        '—': '-', '`': '\'',
        'ʼ': '\''
    }
    replace = re.compile("|".join([re.escape(k) for k in sorted(replacement_punctuation, key=len, reverse=True)]), flags=re.DOTALL)
    word = replace.sub(lambda x: replacement_punctuation[x.group(0)], word)
    extraneous = re.compile(r'^[@#%_=\$\^&\*\+\\]$')
    word = extraneous.sub('', word)
    return word

with open('transcriptions.txt', 'r', encoding='utf-8') as at:
    ttsd = at.readlines()
    allowed_characters_re = re.compile(r'^[a-zịñọʌ!:;"/, \-\(\)\.\'\?ʼ]+$')

    def preprocess_word(word, report=False):
        word = remove_extraneous_punctuation(word)
        if not bool(allowed_characters_re.match(word)):
            if report and word:
                print(f"REPORTING: '{word}'")
            return ''
        return word

    def batch_iterator(batch_size=1000):
        print("Processing ASR texts.")
        for i in range(0, len(ttsd), batch_size):
            yield [preprocess_word(t, True) for t in ttsd[i:i+batch_size]]

    trainer = BpeTrainer(special_tokens=['[STOP]', '[UNK]', '[SPACE]'], vocab_size=255)
    tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    tokenizer.train_from_iterator(batch_iterator(), trainer, length=len(ttsd))
    tokenizer.save('./custom_language_tokenizer.json')

    # Save the tokenizer to hub
    tokenizer.push_to_hub(repo_id='aidystark/ibibio-tokenizer')